In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, sum as _sum, max as _max

spark = SparkSession.builder.appName('CombineCSVs').getOrCreate()

filenames = [
    'combined-property-tax-report-2006-2010.csv',
    'combined-property-tax-report-2011-2015.csv',
    'combined-property-tax-report-2016-2019.csv',
    'combined-property-tax-report-2020.csv'
]

combined_df = None



24/03/29 15:23:28 WARN Utils: Your hostname, joshua-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/03/29 15:23:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/29 15:23:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
for filename in filenames:
    df = spark.read.csv(filename, header=True, inferSchema=True)
    for year in range(2006, 2025):
        df = df.withColumn(f'CURRENT_LAND_VALUE_{year}', when(col('TAX_ASSESSMENT_YEAR') == year, col('CURRENT_LAND_VALUE')).otherwise(None))
        df = df.withColumn(f'CURRENT_IMPROVEMENT_VALUE_{year}', when(col('TAX_ASSESSMENT_YEAR') == year, col('CURRENT_IMPROVEMENT_VALUE')).otherwise(None))
    if combined_df is None:
        combined_df = df
    else:
        combined_df = combined_df.unionByName(df, allowMissingColumns=True)

# Prepare the list of aggregation expressions
aggregations = [
    _max('YEAR_BUILT').alias('YEAR_BUILT'),
    _max('BIG_IMPROVEMENT_YEAR').alias('BIG_IMPROVEMENT_YEAR')
]
aggregations.extend([_sum(col(f'CURRENT_LAND_VALUE_{year}')).alias(f'CURRENT_LAND_VALUE_{year}') for year in range(2006, 2025)])
aggregations.extend([_sum(col(f'CURRENT_IMPROVEMENT_VALUE_{year}')).alias(f'CURRENT_IMPROVEMENT_VALUE_{year}') for year in range(2006, 2025)])

# Perform the aggregation
result_df = combined_df.groupBy('PROPERTY_POSTAL_CODE').agg(*aggregations)

In [3]:

from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName('PostalCodeCountsPerYear').getOrCreate()

filenames = [
    'cleaned-property-tax-report-2006-2010.csv',
    'cleaned-property-tax-report-2011-2015.csv',
    'cleaned-property-tax-report-2016-2019.csv',
    'cleaned-property-tax-report-2020.csv'
]

postal_code_counts = None

for filename in filenames:
    df = spark.read.csv(filename, header=True, inferSchema=True)
    
    # Convert TAX_ASSESSMENT_YEAR to integer to remove decimal point
    df = df.withColumn('TAX_ASSESSMENT_YEAR', F.col('TAX_ASSESSMENT_YEAR').cast('integer'))
    
    # Group by postal code and tax assessment year, and count occurrences
    counts_df = df.groupBy('PROPERTY_POSTAL_CODE', 'TAX_ASSESSMENT_YEAR').count()
    
    # If the aggregated DataFrame is empty, initialize it with the current counts
    if postal_code_counts is None:
        postal_code_counts = counts_df
    else:
        # Union the counts with the existing aggregated data
        postal_code_counts = postal_code_counts.unionByName(counts_df, allowMissingColumns=True)

# Pivot the DataFrame to have one column per tax year with counts for each postal code
pivot_df = postal_code_counts.groupBy('PROPERTY_POSTAL_CODE').pivot('TAX_ASSESSMENT_YEAR').agg(F.first('count'))

# Replace null values with zero in the pivoted DataFrame
pivot_df = pivot_df.na.fill(0)

pivot_df.show()

24/03/29 15:23:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|PROPERTY_POSTAL_CODE|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021|2022|2023|2024|
+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|             V5X 4X1|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|
|             V6H 3S6|   9|   9|  11|  11|  12|  12|  13|  13|  12|  13|   5|   5|   4|   4|   4|   4|   3|   3|   3|
|             V5N 2B9|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|  23|
|             V5M 4J1|   1|   1|   1|   1|   1|   2|   2|   2|   2|   2|   2|   2|   2|   2|   2|   1|   1|   1|   1|
|             V5T 4T5|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|   1|
|             V5W 2H8|  12|  12|  12|  12|  12|  12|  12

In [4]:
# Join the DataFrames on 'PROPERTY_POSTAL_CODE'
enhanced_df = result_df.join(pivot_df, on='PROPERTY_POSTAL_CODE', how='left')

enhanced_df = enhanced_df.fillna(0)

years = list(range(2006, 2025))

# Rename the columns
for year in years:
    enhanced_df = enhanced_df.withColumnRenamed(str(year), f"Count_Per_Year_{year}")

enhanced_df.show()

24/03/29 15:23:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+--------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+-------------------------

In [5]:
enhanced_df.coalesce(1).write.option("header", "true").csv("merged", mode="overwrite")

spark.stop()